# Transcriptomics data analysis

### General module import

In [ ]:
# Load functions
%load_ext autoreload
%autoreload 2
from oleveler import *
# Oleveler: https://gitlab.services.universiteitleiden.nl/ibl-bioinformatic/oleveler
# Set plotting mode for notebook
%matplotlib inline
setDarkModePlotting(forceWhite=True)

# Set logger level. For more info: use logging.INFO
logshandler.setLevel(logging.WARNING)
logshandler.setLevel(logging.INFO)

## Data processing

Data was generated using MaxQuant LFQ method, so we will find 'LFQ intensities' columns from the 'proteinGroups.txt' file.

In [ ]:
# Load data
selfAlignCt = gatherCountTable("selfAlign/gene_counts_filtered/")
saDf = calculateTPM(selfAlignCt, 'selfAlign/gene_counts_filtered/GCF_000203835.1_ASM20383v1_genomic.gff', 
                    tagsForGeneName='locus_tag', removerRNA=True, removeIDcontains=['SCP'])

# Load meta
metaDf, conditions, experiments = loadMeta('Annotation.csv')

# Calculate mean and var
meanDf, nquantDf, varDf, stdDf, semDf = getStats(saDf, experiments)

# Transformation vst using DESeq2
vstDf = deseq2Process(selfAlignCt, metaDf, ref='WT_45')
vstMeanDf, vstNquantDf, vstVarDf, vstStdDf, vstSemDf = getStats(vstDf, experiments, title='vst')

### DESeq2 different expression analysis

In [ ]:
deseq2CompResultsShrink, comparisons = makeCompMatrixDeseq2('comparisons.xlsx', 
                                                      selfAlignCt,
                                                      'annotation.csv',
                                                      shrink='lfcShrink')

## General QC

Both for none transformed and VST transformed data. 

## Data analysis plots

### General analysis

PCA and PLS analysis.

Including volcano plots of different comparisons.

Get the DE from both **DESeq2** and **MSstats** result. The normalisation of MSstats seems not applicable for bad data, be careful about that.

PCA plot VST

In [ ]:
# Remove Gbn from data. The gene has been knockedout from one strain
newVstDf = vstDf[vstDf.index != 'SCO1839']
PcaClass_vst = plotPrincipleAnalysis(newVstDf, colourSet=genColour(vstDf.columns), figsize=(4,4),
                                     title='VST df (no gbn)', analysisType='PCA')
plotPCAExplanation(PcaClass_vst)

In [ ]:
# Loading plot
plotPrincipleAnalysisLoading(vstDf, drawOutliers=True, outlierAlg=0, outliersFraction=0.05, title='VST df')

PLS plot VST

This has similar result as non-supervised PCA.

In [ ]:
plsClasses = [
              [0,24],
              [0,24],
              [0,24],
              [0,45],
              [0,45],
              [0,45],
              [2,24],
              [2,24],
              [2,24],
              [2,45],
              [2,45],
              [2,45],
]
plsClasses = np.array(plsClasses)

PlsClass_vst = plotPrincipleAnalysis(vstDf,
                                     colourSet=genColour(vstDf.columns),
                                     title='VST df',
                                     analysisType='PLS',
                                     plsClasses=plsClasses)

Correlation plot VST

In [ ]:
plotCorr(vstDf, vmin=0.8)

Volcano plot for each comparison.

In [ ]:
c = 'mu_wt24'
# Columns prepared to get the expression data from vstDf
cols = metaDf[(metaDf.Condition==comparisons[c]['exp']) | \
              (metaDf.Condition==comparisons[c]['ctr'])].Experiment.values
# Genes to be highlighted:
targets = OrderedDict([
    ('GlnR regulon' , ['SCO2198', # glnA
            'SCO5583', 'SCO5584', 'SCO5585', # amtB, glnK, glnD
            'SCO2210', # glnII
            'SCO4683', # gdhA
            'SCO2486','SCO2487', 'SCO2488', # nirB_a, nirB_b, nirCD
            'SCO1236', 'SCO1235', 'SCO1234', 'SCO1233',# ureA, B, C, F
            'SCO0255', 'SCO0888', 'SCO2400', 'SCO2404', 'SCO1863', 'SCO2195', 'SCO7155'
           ]),
    (r'$act$',[f'SCO{i}' for i in range(5071, 5093)]),
    (r'$nuo$JKLMN', [f'SCO{i}' for i in range(4571, 4576)]),
    (r'$nuo2$'  ,[f'SCO{i}' for i in range(4599, 4609)]),
    (r"$bxl$AEFG", [f'SCO{i}' for i in range(7028,7032)]),
    ("carboxylesterase related", [f'SCO0{i}' for i in range(319,322)]),
    ("SoxR regulon", ['SCO4265', 'SCO4266','SCO2477','SCO2478', 'SCO7008', 'SCO1909', 'SCO1178']),
    ('Possible HGT cluster',[f'SCO{i}' for i in range(4615, 4628)]),
#     ('Nit' ,[f'SCO{i}' for i in range(5583, 5586)]),
#     ('Nir' ,[f'SCO{i}' for i in range(2486, 2489)]),
#     ('pGlnr' , ['SCO2471', 'SCO2472', 'SCO2473']),
#     ('Act',[f'SCO{i}' for i in range(5071, 5093)]),
#     ('red',[f'SCO{i}' for i in range(5877, 5899)]),
#     ('cpk',[f'SCO{i}' for i in range(6273, 6289)]),
#     ('whi',[f'SCO{i}' for i in [3034,4543,4767,5621,6029,1950,5314,5315,5316,5317,5318,5319,5320,5321,5819]]),
#     ('bld',[f'SCO{i}' for i in [1489,3323,3549,4091,4768,5112,5113,5114,5115,5116,5723,2792]])
])

plotVolcano(deseq2CompResultsShrink[c], 
            vstDf[cols].mean(axis=1), 
            xmax=6, ymax=80,
            highlights=targets,
            figsize=(8,8),
            title=f'shrink_comp_result_{c}')

c = 'mu_wt45'
cols = metaDf[(metaDf.Condition==comparisons[c]['exp']) | \
              (metaDf.Condition==comparisons[c]['ctr'])].Experiment.values
plotVolcano(deseq2CompResultsShrink[c], 
            vstDf[cols].mean(axis=1), 
            highlights=targets,
            figsize=(8,8),
            xmax = 6, ymax=160,
            title=f'shrink_comp_result_{c}')

---

In [ ]:
writeRSessionInfo('RSessionInfo.txt', overwrite=False)